In [ ]:
import json
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime, timedelta, timezone
import requests
import os
from time import sleep
import pytz
from scipy.stats import pearsonr
from Functions import check_correlation, analyze_events, wrap_analyze_events, load_data, data_preparation, wrap_analyze_events_multiprocessing, show_output, start_wrap_analyze_events_multiprocessing
from Functions import get_dynamic_volume_avg, get_benchmark_info, show_output_nomultiprocessing
import pandas as pd
pd.set_option('display.max_rows', None)
from time import time
from multiprocessing import Process
from multiprocessing import Lock, Pool, Manager
from time import time

In [ ]:
start_interval=datetime(2023,5,11)
end_interval=datetime.now()
end_interval=datetime(2023,5,13)


filter_position=(0,100)

data, df, df_obj = load_data(start_interval=start_interval, end_interval=end_interval, filter_position=filter_position)
df

In [ ]:
# WRAP ANALYSIS EVENTS
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.7, 0.8]
list_event_volume = [5,6]

resp = wrap_analyze_events(data, list_buy_vol, list_vol, list_minutes, list_event_buy_volume,
                    list_event_volume, start_interval, end_interval)


In [ ]:
# SHOW DATA FROM WRAP ANALYSIS EVENTS (NO MULTIPROCESSING)
df = show_output_nomultiprocessing(resp)
df

In [ ]:
# WRAP ANALYSIS EVENTS MULTIPROCESSING
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m', 'buy_vol_24h']
list_buy_vol = ['buy_vol_5m', 'buy_vol_15m', 'buy_vol_30m', 'buy_vol_60m']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m', 'vol_24h']
list_vol = ['vol_5m', 'vol_15m', 'vol_30m', 'vol_60m']
list_minutes = [5, 15, 30, 60, 120, 180, 360]
list_event_buy_volume = [0.65, 0.7, 0.8]
list_event_volume = [4,5,6]
n_processes = 6

shared_data = start_wrap_analyze_events_multiprocessing(data, list_buy_vol, list_vol, 
                                                        list_minutes, list_event_buy_volume,
                                                        list_event_volume, start_interval,
                                                        end_interval, n_processes)


In [ ]:
# SHOW DATA FROM MULTIPROCESSING
output = show_output(shared_data)
df = pd.DataFrame(output).transpose()
df.sort_values(by=['lower_bound'], ascending=False)